In [ ]:
import logging
import threading
import itertools
import re
import requests
import json
import os
import gzip
import shutil
from functools import wraps, reduce
from operator import add
from jinja2 import Template
import timeit

# Setup logging
logging.basicConfig(
    filename="library.log",
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

class Book:
    def __init__(self, title, author, year, pages):
        self.title = title
        self.author = author
        self.year = year
        self.pages = pages

    def __repr__(self):
        return f"'{self.title}' by {self.author} ({self.year})"

    def __eq__(self, other):
        if isinstance(other, Book):
            return self.title == other.title and self.author == other.author
        return False

    def __hash__(self):
        return hash((self.title, self.author, self.year))

    def __add__(self, other):
        if isinstance(other, Book):
            return Book(f"{self.title} & {other.title}", self.author, self.year, self.pages + other.pages)
        raise ValueError("Can only add another Book")

    def __str__(self):
        return f"'{self.title}' by {self.author}, {self.pages} pages"

class Library:
    def __init__(self):
        self.books = set()
        self.borrowed_books = {}
        self.lock = threading.Lock()

    def add_book(self, book):
        with self.lock:
            if isinstance(book, Book):
                self.books.add(book)
                logging.info(f"Added book: {book}")
            else:
                raise ValueError("Only Book objects can be added to the library.")

    def remove_book(self, book):
        with self.lock:
            if book in self.books:
                self.books.remove(book)
                logging.info(f"Removed book: {book}")
            else:
                raise ValueError("Book not found in library.")

    def available_books(self):
        return list(filter(lambda book: book not in itertools.chain(*self.borrowed_books.values()), self.books))

    def borrow_book(self, user, book):
        if book in self.available_books():
            with self.lock:
                self.borrowed_books.setdefault(user, []).append(book)
                logging.info(f"{user} borrowed: {book}")
        else:
            raise ValueError(f"'{book.title}' is not available for borrowing.")

    def return_book(self, user, book):
        if user in self.borrowed_books and book in self.borrowed_books[user]:
            with self.lock:
                self.borrowed_books[user].remove(book)
                if not self.borrowed_books[user]:
                    del self.borrowed_books[user]
                logging.info(f"{user} returned: {book}")
        else:
            raise ValueError(f"{user} has not borrowed '{book.title}'.")

    def borrowed_books_by_user(self, user):
        return self.borrowed_books.get(user, [])

    def save_to_file(self, filename="library.json"):
        """Save the library's state to a JSON file."""
        def book_to_dict(book):
            return {
                "title": book.title,
                "author": book.author,
                "year": book.year,
                "pages": book.pages
            }

        with open(filename, "w") as file:
            data = {
                "books": [book_to_dict(book) for book in self.books],
                "borrowed_books": {user: [book_to_dict(book) for book in books] 
                                   for user, books in self.borrowed_books.items()}
            }
            json.dump(data, file)
        logging.info(f"Library saved to {filename}")

    def load_from_file(self, filename="library.json"):
        """Load the library's state from a JSON file."""
        if os.path.exists(filename):
            with open(filename, "r") as file:
                data = json.load(file)
                self.books = {Book(b["title"], b["author"], b["year"], b["pages"]) for b in data["books"]}
                self.borrowed_books = {user: [Book(b["title"], b["author"], b["year"], b["pages"]) 
                                              for b in books] 
                                       for user, books in data["borrowed_books"].items()}
            logging.info(f"Library loaded from {filename}")
        else:
            logging.warning(f"File {filename} does not exist")

    def compress_data(self, filename="library.json", compressed_file="library.json.gz"):
        """Compress the library's JSON file."""
        with open(filename, "rb") as f_in:
            with gzip.open(compressed_file, "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)
        logging.info(f"Compressed {filename} to {compressed_file}")

    def decompress_data(self, compressed_file="library.json.gz", decompressed_file="library_decompressed.json"):
        """Decompress the library's JSON file."""
        with gzip.open(compressed_file, "rb") as f_in:
            with open(decompressed_file, "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)
        logging.info(f"Decompressed {compressed_file} to {decompressed_file}")

    def search_books(self, pattern):
        """Search for books by title using string pattern matching."""
        regex = re.compile(pattern, re.IGNORECASE)
        matches = [book for book in self.books if regex.search(book.title)]
        logging.info(f"Searched for pattern '{pattern}' and found {len(matches)} matches")
        return matches

    def get_book_info_online(self, title):
        """Fetch book information from an online API (dummy example)."""
        try:
            response = requests.get(f"https://openlibrary.org/search.json?title={title}")
            if response.status_code == 200:
                data = response.json()
                if "docs" in data and len(data["docs"]) > 0:
                    book_info = data["docs"][0]
                    logging.info(f"Fetched info for {title}: {book_info}")
                    return book_info
            logging.warning(f"Failed to fetch book info for {title}")
        except requests.RequestException as e:
            logging.error(f"Error fetching book info: {e}")
        return None

def generate_report(library, template_file="report_template.html", output_file="library_report.html"):
    default_template = """
    <html>
    <head>
        <title>Library Report</title>
    </head>
    <body>
        <h1>Library Report</h1>
        <h2>All Books:</h2>
        <ul>
        {% for book in books %}
            <li>{{ book }}</li>
        {% endfor %}
        </ul>
        <h2>Borrowed Books:</h2>
        <ul>
        {% for user, books in borrowed_books.items() %}
            <li>{{ user }}:
                <ul>
                {% for book in books %}
                    <li>{{ book }}</li>
                {% endfor %}
                </ul>
            </li>
        {% endfor %}
        </ul>
    </body>
    </html>
    """
    
    try:
        with open(template_file, 'r') as f:
            template_content = f.read()
    except FileNotFoundError:
        logging.warning(f"Template file {template_file} not found. Using default template.")
        template_content = default_template

    template = Template(template_content)
    rendered_content = template.render(books=library.books, borrowed_books=library.borrowed_books)

    with open(output_file, "w") as f:
        f.write(rendered_content)
    logging.info(f"Generated report: {output_file}")

def filter_books(library, criteria):
    return list(filter(criteria, library.books))

def map_books(library, transform):
    return list(map(transform, library.books))

def total_pages(library):
    return reduce(add, (book.pages for book in library.books), 0)

def measure_performance(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = timeit.default_timer()
        result = func(*args, **kwargs)
        elapsed_time = timeit.default_timer() - start_time
        logging.info(f"Performance of {func.__name__}: {elapsed_time:.6f} seconds")
        return result
    return wrapper

def borrow_books_thread(library, user, book):
    logging.info(f"Thread started for {user} borrowing {book}")
    library.borrow_book(user, book)

@measure_performance
def main():
    library = Library()

    book1 = Book("1984", "George Orwell", 1949, 328)
    book2 = Book("Brave New World", "Aldous Huxley", 1932, 288)
    book3 = Book("The Great Gatsby", "F. Scott Fitzgerald", 1925, 218)
    book4 = Book("The Catcher in the Rye", "J.D. Salinger", 1951, 277)

    for book in [book1, book2, book3, book4]:
        library.add_book(book)

    t1 = threading.Thread(target=borrow_books_thread, args=(library, "User1", book1))
    t2 = threading.Thread(target=borrow_books_thread, args=(library, "User2", book2))
    t1.start()
    t2.start()
    t1.join()
    t2.join()

    library.save_to_file()
    library.compress_data()
    library.decompress_data()

    matching_books = library.search_books(r"great")
    print("Books matching 'great':", matching_books)

    library.get_book_info_online("1984")

    library.borrow_book("User3", book3)

    generate_report(library)

if __name__ == "__main__":
    main()